<a href="https://colab.research.google.com/github/aryannewyork/ALBERT/blob/master/ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.3 MB/s 
     |████████████████████████████████| 182 kB 74.1 MB/s 
     |████████████████████████████████| 7.6 MB 32.3 MB/s 


In [4]:
!git clone https://github.com/huggingface/transformers.git
!pip install -U ./transformers
!pip install git+https://github.com/huggingface/nlp.git

Cloning into 'transformers'...
remote: Enumerating objects: 120425, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 120425 (delta 137), reused 202 (delta 85), pack-reused 120135
Receiving objects: 100% (120425/120425), 114.08 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (89941/89941), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metad

In [5]:
#Activate GPU
# If there is error refer to this: https://forums.developer.nvidia.com/t/nvidia-smi-has-failed-because-it-couldnt-communicate-with-the-nvidia-driver-make-sure-that-the-latest-nvidia-driver-is-installed-and-running/197141
# Download CUDA Drivers from : https://developer.nvidia.com/cuda-downloads
# It will take about 30 mins to download and install, depending upon internet speed
!nvidia-smi

Wed Dec 21 13:25:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 34.2 MB/s 
     |████████████████████████████████| 212 kB 67.9 MB/s 


In [6]:
import torch
import transformers
from transformers import AlbertTokenizerFast
tokenizer=AlbertTokenizerFast.from_pretrained('albert-large-v1')
#train_set=nlp.load_dataset('squad_v2',split='train')

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

In [7]:
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json!

--2022-12-21 13:28:58--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2022-12-21 13:28:58 (293 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2022-12-21 13:28:58--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json!’

sq

In [9]:
import json
from pathlib import Path

def prepare_qa(path):
  path = Path(path)
  with open(path, 'rb') as f:
      squad_dict = json.load(f)

  contexts = []
  questions = []
  answers = []
  for group in squad_dict['data']:
      for passage in group['paragraphs']:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              for answer in qa['answers']:
                  contexts.append(context)
                  questions.append(question)
                  answers.append(answer)

  return contexts, questions, answers

def modify_answer_context(answers,contexts):
  for ans,ctx in zip(answers,contexts):
    
    result_text=ans['text']
    start_idx=ans['answer_start']
    
    end_idx=start_idx+len(result_text)
    if (ctx[start_idx:end_idx]==result_text):
      ans['answer_end']=end_idx
    elif (ctx[start_idx-1:end_idx-1]==result_text):
      ans['answer_start'][0]=start_idx-1
      ans['answer_end']=end_idx-1
    elif (ctx[start_idx-2:end_idx-2]==result_text):
      ans['answer_start'][0]=start_idx-2
      ans['answer_end']=end_idx-2
    else:
      raise ValueError()
train_contexts,train_questions,train_answers=prepare_qa('squad/train-v2.0.json')
val_contexts,val_questions,val_answers=prepare_qa('squad/dev-v2.0.json!')
modify_answer_context(train_answers,train_contexts)
modify_answer_context(val_answers,val_contexts)

In [10]:
train_encode=tokenizer(train_contexts,train_questions,truncation=True,padding=True)
val_encode=tokenizer(val_contexts,val_questions,truncation=True,padding=True)

In [11]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encode, train_answers)
add_token_positions(val_encode, val_answers)

In [12]:
# Training in Pytorch and HF
import torch
from torch.utils.data import Dataset
from transformers import AlbertForQuestionAnswering
from torch.utils.data import DataLoader
from transformers import AdamW

class SquadDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

model=AlbertForQuestionAnswering.from_pretrained("albert-large-v1")

train_dataset = SquadDataset(train_encode)
val_dataset = SquadDataset(val_encode)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)
epochs=2
for epoch in range(epochs):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

Some weights of the model checkpoint at albert-large-v1 were not used when initializing AlbertForQuestionAnswering: ['predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-large-v1 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN

KeyboardInterrupt: ignored

In [13]:
from transformers import  Trainer, TrainingArguments

import torch
from torch.utils.data import Dataset
from transformers import AlbertForQuestionAnswering
from torch.utils.data import DataLoader
from transformers import AdamW

class SquadDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

model=AlbertForQuestionAnswering.from_pretrained("albert-large-v1")

train_dataset = SquadDataset(train_encode)
val_dataset = SquadDataset(val_encode)
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=0.1,              # total number of training epochs
    per_device_train_batch_size=5,  # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=5,
)

#model=AlbertForQuestionAnswering.from_pretrained("albert-large-v1")
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at albert-large-v1 were not used when initializing AlbertForQuestionAnswering: ['predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at albert-large-v1 and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN

Step,Training Loss
5,6.365400
10,6.220600
15,6.001900
20,5.782600
25,5.443900
30,4.926200
35,4.639900
40,4.246900
45,3.952800
50,3.513300


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1737, training_loss=1.4874396302196302, metrics={'train_runtime': 2498.5188, 'train_samples_per_second': 3.475, 'train_steps_per_second': 0.695, 'total_flos': 339657175480320.0, 'train_loss': 1.4874396302196302, 'epoch': 0.1})

In [14]:
from transformers import AutoTokenizer,AutoModelForQuestionAnswering
from transformers import pipeline
model=AutoModelForQuestionAnswering.from_pretrained('abhilash1910/albert-squad-v2')
tokenizer=AutoTokenizer.from_pretrained('abhilash1910/albert-squad-v2')
nlp_QA=pipeline('question-answering',model=model,tokenizer=tokenizer)
QA_inp={
    'question': 'How many parameters does Bert large have?',
    'context': 'Bert large is really big... it has 24 layers, for a total of 340M parameters.Altogether it is 1.34 GB so expect it to take a couple minutes to download to your Colab instance.'
}
result=nlp_QA(QA_inp)
result

Downloading:   0%|          | 0.00/836 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--abhilash1910--albert-squad-v2/snapshots/7ed5f4472cfcfd87279bc0fc7364cccce9374f03/config.json
Model config AlbertConfig {
  "_name_or_path": "abhilash1910/albert-squad-v2",
  "architectures": [
    "AlbertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 16,
  "num_hidden_groups": 1,
  "num_hidden_layers": 24,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version":

Downloading:   0%|          | 0.00/66.6M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--abhilash1910--albert-squad-v2/snapshots/7ed5f4472cfcfd87279bc0fc7364cccce9374f03/pytorch_model.bin
All model checkpoint weights were used when initializing AlbertForQuestionAnswering.

All the weights of AlbertForQuestionAnswering were initialized from the model checkpoint at abhilash1910/albert-squad-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use AlbertForQuestionAnswering for predictions without further training.


Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245 [00:00<?, ?B/s]

loading file spiece.model from cache at None
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--abhilash1910--albert-squad-v2/snapshots/7ed5f4472cfcfd87279bc0fc7364cccce9374f03/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--abhilash1910--albert-squad-v2/snapshots/7ed5f4472cfcfd87279bc0fc7364cccce9374f03/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--abhilash1910--albert-squad-v2/snapshots/7ed5f4472cfcfd87279bc0fc7364cccce9374f03/tokenizer_config.json


{'score': 0.14847153425216675, 'start': 61, 'end': 65, 'answer': '340M'}